In [1]:
import sys
sys.path.insert(0, '..')
from semantic_mpc_interface import (
    ModelBuilder,
    BuildingMetadataLoader,
    add_points,
    SurveyGenerator,
    SurveyReader,
    convert_units,
    BrickToGrafana,
    SHACLHandler
)
from buildingmotif.namespaces import BRICK, RDF
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

# Testing the Model Builder

In [2]:
builder = ModelBuilder(
    site_id="test-site",
    ontology = 's223'
)
builder.add_site(
    timezone="America/New_York",
    latitude=40.7128,
    longitude=-74.0060,
    noaa_station="KJFK",
)

/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))


In [3]:
builder.graph.print()

@prefix bldg: <urn:hpflex/test-site#> .
@prefix hpflex: <urn:hpflex#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix s223: <http://data.ashrae.org/standard223#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

bldg: a owl:Ontology,
        hpflex:Project .

bldg:test-site a s223:PhysicalSpace,
        hpflex:Site ;
    s223:hasProperty bldg:test-site.latitude,
        bldg:test-site.longitude,
        bldg:test-site.noaastation,
        bldg:test-site.timezone .

bldg:test-site.latitude a s223:QuantifiableProperty,
        hpflex:Latitude ;
    s223:hasValue 4.07128e+01 ;
    qudt:hasQuantityKind quantitykind:GeneralizedCoordinate ;
    qudt:hasUnit unit:Degree .

bldg:test-site.longitude a s223:QuantifiableProperty,
        hpflex:Longitude ;
    s223:hasValue -7.4006e+01 ;
    qudt:hasQuantityKind quantitykind:Gener

In [4]:
builder.add_zone("zone1")
builder.add_window("window1", "zone1", area_value=10.5, azimuth_value=180, tilt_value=30, unit = 'FT2')
builder.add_thermostat("tstat1", "zone1", stage_count=2, setpoint_deadband=1, tolerance=2, active=True, resolution = 1)
builder.add_hvac("hvac1", "zone1", cooling_capacity=5.0, heating_capacity=4.0, cooling_cop=3.5, heating_cop=3.0)
builder.add_space("space1", "zone1", area_value=50.0)

builder.add_zone("zone2")
builder.add_window("window2", "zone2", area_value=10.5, azimuth_value=180, tilt_value=30, unit = 'FT2')
builder.add_thermostat("tstat2", "zone2", stage_count=2, setpoint_deadband=1, tolerance=2, active=True, resolution =  1)
builder.add_hvac("hvac2", "zone2", cooling_capacity=5.0, heating_capacity=4.0, cooling_cop=3.5, heating_cop=3.0)
builder.add_space("space2", "zone2", area_value=50.0)

Filling template parameters:  {'name-connection', 'name-target_cp', 'name-name_cp'}
zone1
Filling template parameters:  {'outlet_cp', 'inlet_cp', 'connection'}
Filling template parameters:  {'name-connection', 'name-target_cp', 'name-name_cp'}
zone2
Filling template parameters:  {'outlet_cp', 'inlet_cp', 'connection'}


/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/buildingmotif/dataclasses/template.py:420: UserWarning: Parameters "name-connection, name-target_cp, name-name_cp" were not provided during evaluation
  warnings.warn(
/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/buildingmotif/dataclasses/template.py:420: UserWarning: Parameters "outlet_cp, inlet_cp, connection" were not provided during evaluation
  warnings.warn(


In [5]:
with open('metadata-survey/s223-filled-out-easy-config/point_list.csv') as f:
    point_list = csv.DictReader(f)
    for row in point_list:
        builder.add_point(row['point_name'], row['point_of'], row['point_template'], row['point_type'], row['ref_name'], row['ref_type'], row['unit'])

In [6]:
builder.save_model("test-s223-model.ttl")

# Testing SHACL generation and validation 

In [7]:
og = clone.clone_graph(builder.model.graph)

In [8]:
# Create handler
handler = SHACLHandler()

# Generate shapes
shapes = handler.generate_shapes()

# Save shapes
handler.save_shapes('shapes.ttl')

# Validate a model
conforms, results_graph, results_text = handler.validate_model(builder.model.graph)

builder.model.graph.serialize('test-s223-model-reasoned.ttl', format = 'ttl')
if not conforms:
    print("Validation failed:")
    print(results_text)

In [9]:
# lots of new inferred information
(builder.graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/test-site#connection_2f3c91e6> a ns1:connection .

<urn:hpflex/test-site#connection_fbb9b09e> a ns1:connection .

<urn:hpflex/test-site#hvac1> a ns1:hp-rtu ;
    ns1:air-connects-to <urn:hpflex/test-site#zone1> ;
    ns1:has-point <urn:hpflex/test-site#hvac1_cooling_COP>,
        <urn:hpflex/test-site#hvac1_cooling_capacity>,
        <urn:hpflex/test-site#hvac1_heating_COP>,
        <urn:hpflex/test-site#hvac1_heating_capacity> .

<urn:hpflex/test-site#hvac2> a ns1:hp-rtu ;
    ns1:air-connects-to <urn:hpflex/test-site#zone2> ;
    ns1:has-point <urn:hpflex/test-site#hvac2_cooling_COP>,
        <urn:hpflex/test-site#hvac2_cooling_capacity>,
        <urn:hpflex/test-site#hvac2_heating_COP>,
        <urn:hpflex/test-site#hvac2_heating_capacity> .

<urn:hpflex/test-site#inlet_cp_e3e555e2> a ns1:air-inlet-connectionpoint .

<urn:hpflex/test-site#inlet_cp_e3f060da> a ns1:air-inlet-connectionpoint .

<urn:hpflex/test-site#name-connection_b0343

# Testing get Metadata

In [10]:
site_metadata = BuildingMetadataLoader("test-s223-model-reasoned.ttl", ontology='s223')
site_info = site_metadata.get_site_info()
thermostat_data = site_metadata.get_thermostat_data()

In [11]:
site_info

{'site_id': 'urn:hpflex/test-site#test-site',
 'latitude': 40.7128,
 'longitude': -74.006,
 'NOAAstation': 'KJFK',
 'tz': 'America/New_York',
 'project_id': 'urn:hpflex/test-site#test-site'}

In [12]:
thermostat_data

{'heat_availability': [True, True],
 'cool_availability': [True, True],
 'heat_tolerance': [-2.0, -2.0],
 'cool_tolerance': [2.0, 2.0],
 'setpoint_deadband': [1, 1],
 'active': [True, True],
 'control_group': ['DEPRECATED', 'DEPRECATED'],
 'control_type_list': ['stage', 'stage'],
 'floor_area_list': [50.0, 50.0],
 'window_area_list': [10.5, 10.5],
 'azimuth_list': [180, 180],
 'tilt_list': [30, 30],
 'zone_ids': ['zone1', 'zone2'],
 'hvacs': ['hvac1', 'hvac2'],
 'setpoint_type': ['double', 'double'],
 'fuel_heat_list': ['electricity', 'electricity'],
 'fuel_cool_list': ['electricity', 'electricity'],
 'cooling_capacity': [5.0, 5.0],
 'heating_capacity': [4.0, 4.0],
 'cooling_cop': [3.5, 3.5],
 'heating_cop': [3.0, 3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [1, 1],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_C'),
  rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_C')]}

In [13]:
site_metadata.get_thermostat_data(for_zone='zone1')

{'heat_availability': [True],
 'cool_availability': [True],
 'heat_tolerance': [-2.0],
 'cool_tolerance': [2.0],
 'setpoint_deadband': [1],
 'active': [True],
 'control_group': ['DEPRECATED'],
 'control_type_list': ['stage'],
 'floor_area_list': [50.0],
 'window_area_list': [10.5],
 'azimuth_list': [180],
 'tilt_list': [30],
 'zone_ids': ['zone1'],
 'hvacs': ['hvac1'],
 'setpoint_type': ['double'],
 'fuel_heat_list': ['electricity'],
 'fuel_cool_list': ['electricity'],
 'cooling_capacity': [5.0],
 'heating_capacity': [4.0],
 'cooling_cop': [3.5],
 'heating_cop': [3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [1],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_C')]}

In [14]:
site_metadata.get_complete_output()

{'site_id': 'urn:hpflex/test-site#test-site',
 'latitude': 40.7128,
 'longitude': -74.006,
 'NOAAstation': 'KJFK',
 'tz': 'America/New_York',
 'project_id': 'urn:hpflex/test-site#test-site',
 'heat_availability': [True, True],
 'cool_availability': [True, True],
 'heat_tolerance': [-2.0, -2.0],
 'cool_tolerance': [2.0, 2.0],
 'setpoint_deadband': [1, 1],
 'active': [True, True],
 'control_group': ['DEPRECATED', 'DEPRECATED'],
 'control_type_list': ['stage', 'stage'],
 'floor_area_list': [50.0, 50.0],
 'window_area_list': [10.5, 10.5],
 'azimuth_list': [180, 180],
 'tilt_list': [30, 30],
 'zone_ids': ['zone1', 'zone2'],
 'hvacs': ['hvac1', 'hvac2'],
 'setpoint_type': ['double', 'double'],
 'fuel_heat_list': ['electricity', 'electricity'],
 'fuel_cool_list': ['electricity', 'electricity'],
 'cooling_capacity': [5.0, 5.0],
 'heating_capacity': [4.0, 4.0],
 'cooling_cop': [3.5, 3.5],
 'heating_cop': [3.0, 3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [1, 1],


# Generating Metadata Survey

In [15]:
survey_gen = SurveyGenerator(site_name = 'easy-config-test',
                             hvac_type = 'hp-rtu')
# zone_space_window list for configuration: 3 zones, with 2 spaces and 2 windows, 1 space and 2 windows, 1 space 3 windows, one hvac unit per zone

survey_gen = SurveyGenerator(site_name = 'easy-config-test',
                             hvac_type = 'hp-rtu')

survey_gen.easy_config(
    zone_space_window_list=[(2,2),(1,2),(1,3)], 
    output_path='./metadata-survey')

# survey_gen.easy_config(zone_space_window_list=[(1,1)], output_path='./metadata-survey')


TypeError: SurveyGenerator.__init__() got an unexpected keyword argument 'site_name'

In [ ]:
# Custom config 
# TODO: add underscores to begining of all methods but generate template and easy config
survey_gen = SurveyGenerator(site_name= 'custom-config-test',
                             hvac_type = 'vrf')
survey_gen.generate_template(
    hvacs_feed_hvacs = {'HP1': ['FCU1','FCU2','FCU3'],
                        'HP2': ['FCU1', 'FCU2', 'FCU3']},
    hvacs_feed_zones= {'FCU1': ['Zone1'],
                       'FCU2': ['Zone2'],
                       'FCU3': ['Zone3']},
    zones_contain_spaces= {'Zone1': ['Z1S1', 'Z1S2'],
                           'Zone2': ['Z1S2','Z2S3'],
                           'Zone3': ['Z3S1']},
    zones_contain_windows= {'Zone1': ['W1'],
                           'Zone2': ['W2'],
                           'Zone3': ['W3','W4','W5','W6']},
                           output_path='./metadata-survey'
)

# Reading Metadata Survey

In [ ]:
generator = SurveyReader("./metadata-survey/filled-out-easy-config-test", ontology = 's223')
generator.create_model("survey-brick-model.ttl")

TypeError: ModelBuilder.__init__() got an unexpected keyword argument 'timezone'

In [ ]:
generator.config['hvacs_feed_zones']

{'hvac_1': ['zone_1']}

: 

: 

: 

: 

: 

: 

: 

: 

: 

# Testing Unit Conversion

In [ ]:
convert_units(10, 'FT', 'M')

3.048

: 

: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

: 

: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
convert_units(0, 'DEG_C', 'K')

273.15

: 

: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
site_metadata.convert_model_to_si()
site_metadata.get_thermostat_data(for_zone='zone1')

changing value of  urn:hpflex/test-site#tstat1_resolution from http://qudt.org/vocab/unit/DEG_F to http://qudt.org/vocab/unit/DEG_C
changing value of  urn:hpflex/test-site#tstat1_setpoint_deadband from http://qudt.org/vocab/unit/DEG_F to http://qudt.org/vocab/unit/DEG_C
changing value of  urn:hpflex/test-site#tstat1_tolerance from http://qudt.org/vocab/unit/DEG_F to http://qudt.org/vocab/unit/DEG_C
changing value of  urn:hpflex/test-site#tstat2_resolution from http://qudt.org/vocab/unit/DEG_F to http://qudt.org/vocab/unit/DEG_C
changing value of  urn:hpflex/test-site#tstat2_setpoint_deadband from http://qudt.org/vocab/unit/DEG_F to http://qudt.org/vocab/unit/DEG_C
changing value of  urn:hpflex/test-site#tstat2_tolerance from http://qudt.org/vocab/unit/DEG_F to http://qudt.org/vocab/unit/DEG_C
changing value of  urn:hpflex/test-site#window1_area from http://qudt.org/vocab/unit/FT2 to http://qudt.org/vocab/unit/M2
changing value of  urn:hpflex/test-site#window2_area from http://qudt.org/

{'heat_availability': [False],
 'cool_availability': [True],
 'heat_tolerance': [-1.1111111111111112],
 'cool_tolerance': [1.1111111111111112],
 'setpoint_deadband': [0.5555555555555556],
 'active': [True],
 'control_group': ['urn:hpflex/test-site#tstat1_control_group'],
 'control_type_list': ['stage'],
 'floor_area_list': [50.0],
 'window_area_list': [0.9754819200000001],
 'azimuth_list': [180],
 'tilt_list': [30],
 'zone_ids': ['zone1'],
 'hvacs': ['hvac1'],
 'setpoint_type': ['double'],
 'fuel_heat_list': ['gas'],
 'fuel_cool_list': ['electricity'],
 'cooling_capacity': [5.0],
 'heating_capacity': [4.0],
 'cooling_cop': [3.5],
 'heating_cop': [3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [0.5555555555555556],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F')]}

: 

: 

: 

...

: 

: 

: 

: 

: 

In [ ]:
"""
The temperature unit still shows up as DEG_F, because it is at the units for the zone temperature, 
which does not have a value to convert in the model, it's values are in the DB.
it is a reminder to also convert that to C also. We can use the unit_conversion script for any conversions in the db.
"""

"\nThe temperature unit still shows up as DEG_F, because it is at the units for the zone temperature, \nwhich does not have a value to convert in the model, it's values are in the DB.\nit is a reminder to also convert that to C also. We can use the unit_conversion script for any conversions in the db.\n"

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

# Testing Grafana Dashboarding 

In [ ]:
import yaml 
with open('../development_files/creds.yml') as f:
    config = yaml.safe_load(f)

bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

: 

: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
bg.create_dashboard('AQL from brick')

Dashboard(title='AQL from brick', annotations=Annotations(list=[]), description='Zone-based dashboard generated from Brick model', editable=True, gnetId=None, graphTooltip=0, hideControls=False, id=None, inputs=[], links=[], panels=[TimeSeries(dataSource='AQLCentral', targets=[SqlTarget(expr='', format='time_series', hide=False, legendFormat='', interval='', intervalFactor=2, metric='', refId='Speed_Status', step=10, target='', instant=False, datasource='AQLCentral', rawSql='SELECT \n                ts AS "time", \n                CAST(value_string AS FLOAT) AS "Speed_Status"\n            FROM \n                data \n            WHERE \n                topic_id in (select topic_id from topics where topic_name = \'aql/1610101/vfd_output\')\n            AND \n                $__timeFilter(ts) \n            ORDER BY 1', rawQuery=True, srcFilePath='', sqlParams={}), SqlTarget(expr='', format='time_series', hide=False, legendFormat='', interval='', intervalFactor=2, metric='', refId='Unocc

: 

: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
bg.upload_dashboard(message = 'testing upload')

Upload status: 200 - b'{"id":10,"slug":"aql-from-brick","status":"success","uid":"d61fc704-4ac4-415e-9ee1-a6a270f5866c","url":"/d/d61fc704-4ac4-415e-9ee1-a6a270f5866c/aql-from-brick","version":4}'


: 

: 

: 

: 

: 

: 

: 

: 

: 